In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
df = pd.read_csv("data/Madrid/listings-detailed-mad.csv")
print(df.head())

                    id                                       listing_url  \
0  1104967621421098322  https://www.airbnb.com/rooms/1104967621421098322   
1   930941073152690361   https://www.airbnb.com/rooms/930941073152690361   
2             51647382             https://www.airbnb.com/rooms/51647382   
3  1026072412626070839  https://www.airbnb.com/rooms/1026072412626070839   
4   766423030884644417   https://www.airbnb.com/rooms/766423030884644417   

        scrape_id last_scraped       source                              name  \
0  20240615011404   2024-06-15  city scrape  Habitación de lujo, baño privado   
1  20240615011404   2024-06-15  city scrape            habitación para pareja   
2  20240615011404   2024-06-15  city scrape            Buenos Aires en Madrid   
3  20240615011404   2024-06-15  city scrape            Habitación Madrid Rio!   
4  20240615011404   2024-06-15  city scrape           habitación para turista   

                                         description  ne

In [4]:
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [5]:
df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 75, dtype: object

In [6]:
# Count total IDs in df
df['host_id'].count()

26924

In [7]:
# Count unique IDs in df
df['host_id'].nunique()

12834

In [8]:
# See unique values in room_type column
df['room_type'].unique()

array(['Private room', 'Shared room', 'Entire home/apt', 'Hotel room'],
      dtype=object)

In [9]:
# Count number of each value in room_type column
df["room_type"].value_counts()

room_type
Entire home/apt    17346
Private room        9103
Shared room          381
Hotel room            94
Name: count, dtype: int64

In [10]:
# View the above as percentages of the total
df["room_type"].value_counts(normalize=True) * 100

room_type
Entire home/apt    64.425791
Private room       33.809984
Shared room         1.415094
Hotel room          0.349131
Name: proportion, dtype: float64

In [11]:
# See unique values in neighbourhood column
df['neighbourhood'].unique()

array([nan, 'Madrid, Comunidad de Madrid, Spain',
       'Madrid, Community of Madrid, Spain', 'Madrid, Spain',
       'Chueca, Madrid, Spain', 'Madrid , Comunidad de Madrid, Spain',
       'MADRID, Spain', 'Madrid, Carabanchel alto, Spain',
       'Madrid, Co, Spain', 'Madrid , Madrid, Spain',
       'Madrid Center, Madrid, Spain',
       'Pozuelo de Alarcón, Comunidad de Madrid, Spain',
       'Madrid, SPAIN  Madrid Madrid, Spain',
       'Madrid, Madrid, Barajas, Spain',
       'Madrid, Comunidad de Madrid, ES, Madrid, Spain',
       'ROSES (ROSAS ) ESPAGNE, ROSES, Spain',
       'Madrid, Palos de Moguer , Spain',
       'Madrid, Comunidad de Madrid, España, Spain', 'Spain',
       'Madrid, Centro Madrid, Spain',
       'Madrid, Spain Madrid Madrid, Spain', 'Madrid, MADRID, Spain',
       'Madrid , Comunidad de Madrid, España, Spain',
       'Madrid, Comunidad de  Madrid, Spain', 'Salamanca, Madrid, Spain',
       'Madrid, Comunidad de Madrid , Atocha, Spain',
       'Comunidad de M

In [12]:
# Count number of each value in neighbourhood column
df["neighbourhood"].value_counts()

neighbourhood
Madrid, Comunidad de Madrid, Spain                           12419
Madrid, Community of Madrid, Spain                             465
Madrid, Spain                                                  335
Madrid, Co, Spain                                               22
Madrid Center, Madrid, Spain                                    10
MADRID, Spain                                                    9
Madrid , Madrid, Spain                                           7
Chueca, Madrid, Spain                                            5
Madrid , Comunidad de Madrid, Spain                              3
Pozuelo de Alarcón, Comunidad de Madrid, Spain                   3
Madrid, Comunidad de Madrid , Atocha, Spain                      2
Madrid, Comunidad de  Madrid, Spain                              2
Spain                                                            2
Madrid, Comunidad de Madrid, España, Spain                       2
Madrid, Comunidad de Madrid, ES, Madrid, Spain  

In [13]:
#Crear nuevo dataset con menos columnas

# Creating the new DataFrame with the required columns
listings_prepared_mad = df[['host_id', 'host_name', 'calculated_host_listings_count', 
                            'host_is_superhost', 'listing_url', 'id', 'name', 
                            'neighbourhood_group_cleansed', 'latitude', 'longitude', 
                            'room_type', 'bedrooms', 'price', 'number_of_reviews', 
                            'review_scores_rating', 'license']].copy()

# Adding 'kitchen', 'patio or balcony', 'elevator', 'air conditioning' columns based on 'amenities'
listings_prepared_mad['kitchen'] = df['amenities'].str.contains('kitchen', case=False, na=False).astype(int)
listings_prepared_mad['patio or balcony'] = df['amenities'].str.contains('patio|balcony', case=False, na=False).astype(int)
listings_prepared_mad['elevator'] = df['amenities'].str.contains('elevator', case=False, na=False).astype(int)
listings_prepared_mad['air conditioning'] = df['amenities'].str.contains('air conditioning', case=False, na=False).astype(int)

# Adding 'long_term', 'short_term', 'possible_long_term' columns based on 'minimum_nights' and 'maximum_nights'
listings_prepared_mad['long_term'] = (df['minimum_nights'] > 31).astype(int)
listings_prepared_mad['short_term'] = (df['maximum_nights'] <= 31).astype(int)
listings_prepared_mad['possible_long_term'] = ((df['minimum_nights'] >= 1) & (df['maximum_nights'] >= 31)).astype(int)

# Display the first few rows of the new dataframe
print(listings_prepared_mad.head())


     host_id        host_name  calculated_host_listings_count  \
0  565307927         Carolina                               1   
1  442944608   Jonathan Jesus                               2   
2  252336445            Norma                               2   
3  545435841       Gean Piero                               1   
4    3882456  Gabriela CHÁVEZ                               1   

  host_is_superhost                                       listing_url  \
0                 f  https://www.airbnb.com/rooms/1104967621421098322   
1                 f   https://www.airbnb.com/rooms/930941073152690361   
2                 f             https://www.airbnb.com/rooms/51647382   
3                 f  https://www.airbnb.com/rooms/1026072412626070839   
4                 f   https://www.airbnb.com/rooms/766423030884644417   

                    id                              name  \
0  1104967621421098322  Habitación de lujo, baño privado   
1   930941073152690361            habitación para 

In [15]:
# Export the DataFrame to an Excel file
listings_prepared_mad.to_excel('listings_prepared_mad.xlsx', index=False)

In [14]:
# Reordering the columns
ordered_columns = [
    'id', 'listing_url', 'name', 'host_id', 'host_name', 'calculated_host_listings_count', 
    'host_is_superhost', 'license', 'neighbourhood_group_cleansed', 'latitude', 
    'longitude', 'room_type', 'bedrooms', 'price', 'kitchen', 'patio or balcony', 
    'elevator', 'air conditioning', 'long_term', 'short_term', 'possible_long_term', 
    'number_of_reviews', 'review_scores_rating'
]

# Apply the new order to the DataFrame
listings_prepared_mad = listings_prepared_mad[ordered_columns]

# Display the first few rows of the reordered DataFrame
print(listings_prepared_mad.head())

# Export the reordered DataFrame to a CSV file
listings_prepared_mad.to_csv('listings_prepared_mad.csv', index=False)

                    id                                       listing_url  \
0  1104967621421098322  https://www.airbnb.com/rooms/1104967621421098322   
1   930941073152690361   https://www.airbnb.com/rooms/930941073152690361   
2             51647382             https://www.airbnb.com/rooms/51647382   
3  1026072412626070839  https://www.airbnb.com/rooms/1026072412626070839   
4   766423030884644417   https://www.airbnb.com/rooms/766423030884644417   

                               name    host_id        host_name  \
0  Habitación de lujo, baño privado  565307927         Carolina   
1            habitación para pareja  442944608   Jonathan Jesus   
2            Buenos Aires en Madrid  252336445            Norma   
3            Habitación Madrid Rio!  545435841       Gean Piero   
4           habitación para turista    3882456  Gabriela CHÁVEZ   

   calculated_host_listings_count host_is_superhost license  \
0                               1                 f     NaN   
1             